<a href="https://colab.research.google.com/github/ashu5644/audio_workspace/blob/main/librosa_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install librosa==0.10.0
import librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0.post2
    Uninstalling librosa-0.10.0.post2:
      Successfully uninstalled librosa-0.10.0.post2


**Audio Loading**

In [3]:
import numpy as np
!wget https://voiceage.com/wbsamples/in_mono/Conference.wav
y, sr = librosa.load('Conference.wav', sr=16000, mono=True, dtype=np.float32, offset=1, duration=1)
print(y.shape, sr)

--2023-05-24 06:04:59--  https://voiceage.com/wbsamples/in_mono/Conference.wav
Resolving voiceage.com (voiceage.com)... 24.37.80.233
Connecting to voiceage.com (voiceage.com)|24.37.80.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 472488 (461K) [audio/x-wav]
Saving to: ‘Conference.wav’

Conference.wav      100%[===================>] 461.41K  2.77MB/s    in 0.2s    

2023-05-24 06:04:59 (2.77 MB/s) - ‘Conference.wav’ saved [472488/472488]

(16000,) 16000


In [4]:
librosa.get_samplerate('Conference.wav')

48000

In [5]:
#librosa.stream

In [8]:
#!wget https://voiceage.com/wbsamples/in_stereo/Conference.wav
y, sr = librosa.load('Conference.wav.1', mono=False)
print(y.shape, sr)

(2, 113807) 22050


In [9]:
y_mono = librosa.to_mono(y)
print(y_mono.shape)

(113807,)


In [11]:
duration = librosa.get_duration(y=y_mono, sr=22050)
duration

5.161315192743764

In [12]:
sr = librosa.get_samplerate('Conference.wav.1')
sr

48000

In [13]:
y_16k = librosa.resample(y_mono, orig_sr=22050, target_sr=16000)
print(y_16k.shape)

(82582,)


In [18]:
# 25 ms , 10 ms hop
sr = librosa.get_samplerate('Conference.wav')
hop_length=480
frame_length=1200
block_length=10
stream_gen = librosa.stream(path='Conference.wav', frame_length=frame_length, hop_length=hop_length, block_length=block_length)
for block in stream_gen:
  print(block.shape)
  break
total_len_per_block = (hop_length*(block_length-1)+frame_length)
total_len_per_block

(5520,)


5520

**Time Doamin Processing**

In [ ]:
'''
 AutoCorrelation
  Autocorrelation is a mathematical operation that measures the similarity between a signal and a time-delayed version of itself. 
  It is commonly used in signal processing and audio analysis tasks to identify repeating patterns or periodic components within a signal.
  Process:
    1. Caluclate mean value of amplitude array and substract from singal.
    2. Using FFT calculate autocorrelation at different time delays.
  Usecases:
    1. Pitch estimation, Tempo detection, Harmonic analysis, identifying perodicity, fundamental frequency, rethymic patterns, melody extraction, instrument recognition
  
  Pitch:
    Pitch is a subjective perception of fundamental frequency of a signal.
    Higher fundamental frequncy corrosponds to higher pitch and vice versa.
    Pitch is a perceptual phenomenon, it can vary among individuals based on timbre and harmonics.
    Using fundamental frequency we can give an estimate of pitch of a signal.
    It can be calculated by using autocorrelation function as well
  fundamental frequency:
    The fundamental frequency represents the lowest frequency component of a complex sound.

  timbre:

  harmonics:

  Tempo:
    Tempo refers to the perceived speed or rhythm of a musical piece, typically measured in beats per minute (BPM). 

  Zero-Crossing Rate:
     in general, a higher zero crossing rate indicates a higher frequency or pitch, while a lower zero crossing rate suggests a lower frequency or pitch.
     Rapid changes in the zero crossing rate often indicate the beginning of a sound or note.
     By analyzing the distribution and density of zero crossings, segments with distinct characteristics, such as silence or different sounds, can be identified.

The mu-law algorithm :
    It is used in compression, quantization, expansion of an audio signal to preserve maxium quality during these operations instead of going via standard techniques.
    

In [34]:
# Pitch Estimation
y, sr = librosa.load(librosa.ex('trumpet'), sr=None)
print(y.shape, sr)
autocorr = librosa.autocorrelate(y)
print(autocorr.shape)
lag = autocorr.argmax() # period of repeating waveform signal
print(lag)
pitch_frequency = sr / lag
print(pitch_frequency)

# Zero-Crossing
zc = librosa.zero_crossings(y)
zc
# indices of zero crossings
np.nonzero(zc)

(117601,) 22050
(117601,)
0
inf


<ipython-input-34-5c90ff77a50c>:8: RuntimeWarning: divide by zero encountered in long_scalars
  pitch_frequency = sr / lag


(array([     0,      3,      5, ..., 117598, 117599, 117600]),)

In [35]:
import IPython
IPython.display.Audio(y, rate=sr)

In [36]:
import IPython
y_compress = librosa.mu_compress(y)
IPython.display.Audio(y_compress, rate=sr)

In [37]:
import IPython
y_expand = librosa.mu_expand(y_compress)
IPython.display.Audio(y_expand, rate=sr)

In [38]:
from sys import getsizeof
getsizeof(y), getsizeof(y_compress), getsizeof(y_expand)

(112, 940920, 940920)